In [ ]:
require 'torch'
require 'nn'
require 'image'
mnist = require 'mnist'

In [ ]:
-- Whether to use CUDA, -1: use CPU, >=0: use corresponding GPU
gpuid = 0
if gpuid >= 0 then
    use_cuda = true
    print('using CUDA on GPU ' .. gpuid .. '...')
    require 'cutorch'
    require 'cunn'
    cutorch.setDevice(gpuid + 1) -- note +1 to make it 0 indexed! sigh lua
end

In [ ]:
state = mnist.load('checkpoints/mnist-2015-6-17-22-55-0.9877.nn', use_cuda)
print ('params: ', state.dog.params:type(), state.dog.params:storage():size())
print (string.format('mean: %s, std: %s', state.mean, state.std))

In [ ]:
digits = image.load('data/digits.png', 1, 'byte')
digits = image.scale(digits, 320, 320)
digits = digits:float()
digits:add(-state.mean)
digits:div(state.std)
print(string.format('digits mean: %s', digits:mean()))
itorch.image(digits)

In [ ]:
good_cnt = 0
bad_cnt = 0
for i = 1, 10 do
  for j = 1, 10 do
    local x = digits:narrow(2, (i-1)*32 + 1, 32):narrow(3, (j-1)*32 + 1, 32)
    itorch.image(x)
    local num = state:predict(x:reshape(1, 1, 32, 32))[1][1]-1
    if num == i % 10 then
        good_cnt = good_cnt + 1
        print(string.format('%s - OK', num))
    else
        bad_cnt = bad_cnt + 1
        print(string.format('%s - error!', num))
    end
  end
end
print(string.format('Accuracy: %s%%', good_cnt))
print(string.format('Error: %s%%', bad_cnt))